In [3]:
import numpy as np

# Value Loss

In [20]:
def r_gamma(rewards: np.array, gamma):
    r_gamma = 0
    for result in rewards[:0:-1]:  # Slicing to reverse except the first element
        r_gamma = gamma * (r_gamma + result)
        print(result)
    r_gamma += rewards[0]
    return r_gamma

def value_function(state): #TODO: implement this
    return 0

def value_loss(r_gamma, state, deltas):
    return (np.clip(r_gamma, -deltas[1], deltas[2])-value_function(state))**2

# Policy Loss

In [ ]:
def ratio(old_policy, new_policy, action, state):
    return new_policy(action, state) / old_policy(action, state)

def a_gae(results, states, value_function: callable, gamma, lambda_):
    def a_k(k):
        a_k = - value_function(states[0]) # First state of the hand
        for i in range(k):
            a_k += gamma**(i) * results[i]
        a_k += gamma**(k) * value_function(states[k])
        return a_k
    a_ks = [lambda_**(k_minus_one) * a_k(k_minus_one+1) for k_minus_one in range(len(results)-2)]
    return (1 - lambda_) * np.sum(a_ks)
    
def tc_loss_function(ratio, advantage, epsilon, deltas): #We compute this for every hand and then average it
    return np.clip(ratio, np.clip(ratio, 1 - epsilon, 1 + epsilon), deltas[1]) * advantage
    

In [21]:
rewards = np.array([-20, -40, 0, -100, 320])
deltas = [1, 160, 160]
print(r_gamma(rewards, 0.999))
print(value_loss(r_gamma(rewards, 0.999), 0, deltas))

320
-100
0
-40
159.06161882032
320
-100
0
-40
25300.598581740778
